In [ ]:
import numpy as np

import os
from datetime import datetime
import glob
from multiprocessing import Pool

from scipy.ndimage import gaussian_filter

from uncertainty import ComputeSidUncertainty

%matplotlib inline

In [ ]:
# filter size (Gaissian filter was 5, but weights are smaller for larger distances)
size = 3

# should be 2210 to cover 6 years
n_steps = 2210 

sia_dir = 'NERSC_arctic25km_sea_ice_age_v2p1'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
sid_dir = 'OSISAF_ice_drift_CDR_postproc'
ifiles = sorted(glob.glob(f'{sid_dir}/*/ice_drift*npz'))
idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]
print(f'Found {len(ifiles)} ice drift files from {idates[0]} to {idates[-1]}')

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']

# Accumulate and save SID uncertainty
$\sigma_{i} = \sqrt{<\sigma_{i-1}^2> + \sigma_S^2}$

$\sigma_S$ - uncertainty of smoothed SID from CDR

$\sigma_{i-1}$ - uncertainty from previous step

$<>$ - advection operation

$\sigma_S = \frac{\sqrt{\sum_j^N{\sigma_{Oj}}}}{\sqrt{N}}$

$\sigma_{Oj}$ - uncertaimty in neighbour $j$

$N$ - number of neighbours

In [ ]:
start_indices = [i for i,j in enumerate(idates) if j.year > 2018 and j.month == 9 and j.day == 5]

In [ ]:
csidu = ComputeSidUncertainty(ifiles, idates, mesh_dir, unc_dir, mesh_init_file, n_steps, size, xc, yc)
with Pool(2) as p:
    p.map(csidu.compute_uncertainty, start_indices)